In [1]:
import dash
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output, State
import numpy as np

In [2]:
jobsDF = pd.read_csv('../data/Egypt Job Posts 22 Preprocessed.csv')
jobsDF

,Title,Company Name,Governorate,Country,Job Type,Experience Level,Min Yrs of Excperience,Max Yrs of Experience,Source,Skills - 1,...,Skills - 1.3,Skills - 2,Skills - 2.1,Skills - 3,Skills - 3.1,Skills - 3.2,Skills - 3.3,Skills - 4,Skills - 5,Skills - 6
0,Banking Call Center Agent,Raya Customer Experience,Cairo,Egypt,Full Time,Entry Level,0.0,5.0,Wuzzuf,Banking,...,Customer Care,Call Center,Customer Service,Customer Support,None,None,None,None,None,None
1,Call Center Agent Banking,Raya Customer Experience,Maadi,Egypt,Full Time,Entry Level,0.0,5.0,Wuzzuf,Banking,...,Call Center,Customer Care,Customer Service,English,None,None,None,None,None,None
2,Odoo Functional Consultant,Centione,Nasr City,Egypt,Full Time,Experienced,2.0,5.0,Wuzzuf,Accounting,...,Implementation,ERP,Financial Management,Project Management,None,None,None,None,None,None
3,CRM Specialist,Fawry Microfinance,Cairo,Egypt,Full Time,Entry Level,1.0,2.0,Wuzzuf,Accounting,...,None,None,None,None,None,None,None,None,None,None
4,Core Banking Technical Support Engineer,Confidential,Giza,Egypt,Full Time,Experienced,3.0,6.0,Wuzzuf,Banking,...,IT/Software Development,Engineering,Computer Science,Software Development,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11808,WMS Specialist,Milezmore,Giza,Egypt,Full Time,Entry Level,1.0,3.0,Wuzzuf,Logistics,...,ERP,None,ERP,None,None,None,None,None,None,None
11809,Workforce Specialist,Breadfast,Cairo,Egypt,Full Time,Entry Level,1.0,3.0,Wuzzuf,Customer Service,...,None,None,None,None,None,None,None,None,None,None
11810,Workforce Supervisor,Andalusia Group,Cairo,Egypt,Full Time,Experienced,3.0,5.0,Wuzzuf,Customer Service,...,None,None,None,None,None,None,None,None,None,None
11811,Workforce Supervisor,Majorel Egypt,Cairo,Egypt,Full Time,Experienced,2.0,3.0,Wuzzuf,Customer Service,...,None,None,None,None,None,None,None,None,None,None


In [3]:
#Bans


In [4]:
app = Dash(
    external_stylesheets=[dbc.themes.DARKLY], update_title='Loading...'
)

In [5]:
#Side Bar Code
# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "width": "25rem",
    "background-color": "#222",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

DROPDOWN_STYLE = {
    "background-color": "#222",
    "color": "#222",
    # "color": "#ffffff",
    "fontColor": "#222",
    "text-color": "#222",
    "font-family": "sans-serif",
}


minYearsMin = jobsDF['Min Yrs of Excperience'].min()
minYearsMax = jobsDF['Min Yrs of Excperience'].max()
maxYearsMin = jobsDF['Max Yrs of Experience'].min()
maxYearsMax = jobsDF['Max Yrs of Experience'].max()

sideBar = html.Div([
            html.Div([
                html.H3("Filter", className="display-4"),
                html.Hr(),
            ]),
        dbc.Nav(
            [
                html.Div([
                    html.H4('Analysis of Jobs'),
                    dcc.Dropdown(
                        id="dropdownGraph1_id",
                        options=[{'label' : str(i), 'value' : str(i)} for i in jobsDF['Experience Level'].unique()],
                        value=None,
                        placeholder='Choose an Experience Level....',
                        multi=True, 
                        style=DROPDOWN_STYLE                        
                    ),
                    html.Div([
                        html.Br(),
                        html.H4('Choose The Min Years of Experience'),
                        dcc.Slider(id='slider1_id', min=minYearsMin, max=minYearsMax, marks={str(i): str(i) for i in jobsDF['Min Yrs of Excperience'].unique()}, step=None, value=minYearsMin)
                    ]),
                    html.Div([
                        html.Br(),
                        html.H4('Choose The Max Years of Experience'),
                        dcc.Slider(id='slider2_id',min=maxYearsMin, max=maxYearsMax, marks={str(i): str(i) for i in jobsDF['Max Yrs of Experience'].unique()}, step=None)
                    ]),
                ])
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="sidebar-content", style=CONTENT_STYLE)


In [6]:
#Title Code
TITLE_STYLE = {
    "position": "flex",
    "top": 0,
    "padding": "2rem 2rem",
}

dashboardTitle = html.Div([
                html.H1("Egypt Job Posts 2022", className="twelve columns", style={"text-align": "center"})
    ], style=TITLE_STYLE,  className = 'row', 
)

In [7]:
GRAPH_STYLE = {
    "width": "30rem",
}

graph1 = html.Div([
        html.H4('Analysis of Jobs Titles using bar plot'),
        dcc.Graph(id="graph1_id"),
    ], 
)


@app.callback(
            Output(component_id='graph1_id', component_property='figure'),
            Input(component_id='dropdownGraph1_id', component_property='value')
)
def update_bar_chart(dropdown_value):
    if dropdown_value == None:
        fig = px.bar(jobsDF, x='Skills - 1', y='Title', hover_name='Experience Level', color='Experience Level')
    else:
        filteredDF = jobsDF[jobsDF['Title'].isin(dropdown_value)]
        fig = px.bar(filteredDF, x='Skills - 1', y='Title', hover_name='Experience Level', color='Experience Level')
    return fig



In [8]:
#Graph 2
graph2 = html.Div([
        html.H4('Analysis of Jobs Experience Level using bar'),
        dcc.Graph(id="graph2_id"),
    ], 
)


@app.callback(
            Output(component_id='graph2_id', component_property='figure'),
            Input(component_id='dropdownGraph1_id', component_property='value')
)
def update_scatter_chart(dropdown_value):
    if dropdown_value == None:
        fig = px.bar(jobsDF, x='Company Name', y='Experience Level', width=400, height=400, hover_name='Experience Level', color='Experience Level', barmode='group')
    else:
        filteredDF = jobsDF[jobsDF['Experience Level'].isin(dropdown_value)]
        fig = px.bar(filteredDF, x='Company Name', y='Experience Level', width=400, height=400, hover_name='Experience Level', color='Experience Level', barmode='group')
    return fig

In [9]:
#cards
CARD_STYLE={
    "width": "18rem",
    "margin-bottom": "2rem",
}

card1 = dbc.Card(
    [
        
        dbc.CardBody(
            html.P("This card 1", className="card-text", style={"text-align": "center"})
        ),
    ],
    style=CARD_STYLE,
)

card2 = dbc.Card(
    [
        dbc.CardBody(
            html.P("This card 2", className="card-text", style={"text-align": "center"})
        ),
        
    ],
    style=CARD_STYLE,
)

card3 = dbc.Card(
    [
        dbc.CardBody(
            html.P("This card 3", className="card-text", style={"text-align": "center"})
        ),
        
    ],
    style=CARD_STYLE,
)

card4= dbc.Card(
    [
        dbc.CardBody(
            html.P("This card 4", className="card-text", style={"text-align": "center"})
        ),
        
    ],
    style=CARD_STYLE,
)

In [10]:
app.layout = html.Div([
            dashboardTitle, 
    
            dbc.Row([
                dbc.Col(card1, width="auto"),
                dbc.Col(card2, width="auto"),
                dbc.Col(card3, width="auto"),
                dbc.Col(card4, width="auto"),
            ]), 
    
            dbc.Row([
                dbc.Col(sideBar, width="auto"),
                dbc.Col(graph1, width="auto"),
                dbc.Col(graph2, width="auto"),
                
            ]),
            dbc.Row([
                    
            ],justify="end"),
    
                   
        ])

In [11]:
help(px.bar)

Help on function bar in module plotly.express._chart_types:

bar(data_frame=None, x=None, y=None, color=None, pattern_shape=None, facet_row=None, facet_col=None, facet_col_wrap=0, facet_row_spacing=None, facet_col_spacing=None, hover_name=None, hover_data=None, custom_data=None, text=None, base=None, error_x=None, error_x_minus=None, error_y=None, error_y_minus=None, animation_frame=None, animation_group=None, category_orders=None, labels=None, color_discrete_sequence=None, color_discrete_map=None, color_continuous_scale=None, pattern_shape_sequence=None, pattern_shape_map=None, range_color=None, color_continuous_midpoint=None, opacity=None, orientation=None, barmode='relative', log_x=False, log_y=False, range_x=None, range_y=None, text_auto=False, title=None, template=None, width=None, height=None)
        In a bar plot, each row of `data_frame` is represented as a rectangular
        mark.
        
    Parameters
    ----------
    data_frame: DataFrame or array-like or dict
        

In [ ]:
app.title = 'Egypt Job Posts 2022'
if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Apr/2022 14:28:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2022 14:28:34] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_3_0m1647776574.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2022 14:28:34] "GET /_dash-component-suites/dash/deps/react@16.v2_3_0m1647776574.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2022 14:28:35] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_3_0m1647776574.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2022 14:28:35] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_3_0m1647776574.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2022 14:28:35] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_0_3m1648451819.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2022 14:28:35] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_3_0m1647776574.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2022 14

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Shrook\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Shrook\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Shrook\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Shrook\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Shrook\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Shrook\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Shrook\Anaconda3\lib\site-packages\dash\dash.py", line 1345, in dispatch
    respons

127.0.0.1 - - [23/Apr/2022 14:28:35] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [23/Apr/2022 14:28:37] "POST /_dash-update-component HTTP/1.1" 200 -
